# Single-Site Simulations
These simulations were presented in [Sorojsrisom et al. (2022)](https://bsapubs.onlinelibrary.wiley.com/doi/full/10.1002/aps3.11472). Simulations test dynamics of a single beneficial mutation at a single site in the genome when site is expressed in gametophyte (haploid) life stage only vs. sporophyte (diploid) life stage only. 

In [3]:
import shadie

## Setting up the Simulations

### Make Custom Mutations
First, we create two mutation types with differential expression - one mutation type will only affect the haploid life stage and the other will only affect the diploid life stage. 

In [4]:
m3 = shadie.mtype(0.5, 'f', 0.5, diffexpr="haploid") #introduced beneficial mutation in haploids
m4 = shadie.mtype(0.5, 'f', 0.1, diffexpr="diploid") #introduced beneficial mutation in diploids
NEUT = shadie.mtype(0.5, "f", 0.0,) #neutral mutations in the rest of the chromosome

### Make Custom Genomic Elements
We only need a single genomic element for this simple simulation. We will set neutral mutations to occur 100% of the time, because the other two mutation types (m3 and m4) will be introduced manually at a specific time. 

In [8]:
e1 = shadie.etype([NEUT, m3, m4], [1,0,0]) #Neutral mutations occur 100% of the time

### Build the Chromosome
This genomic element will define the entire chromosome, which is essentially a neutral chromosome

In [10]:
chrom = shadie.chromosome.explicit(data = {(0, 5000000): e1},
                                   use_synonymous_sites_in_coding=False,
                                   )

When we visualize our chromosome we see one solid block of coding region, because m3 and m4 mutations types *could* happen anywhere on the chromosome, but the chance that SLiM will introduce either mutation is 0

In [11]:
chrom.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2bc7eef59e5947a194e872b11c02e50c"> name: None
interval: (0, 5000000)
ElementType: g7
coding: True 0 1000000 2000000 3000000 4000000 5000000

### Overview
This simulation will be loaded from a 10,000 diploid generation burn-in. The simulation will run for an additional 1,000 diploid generations, then add a single beneficial mutation (selection coefficient = 0.5) to the gene expressed in the gametophyte and a single beneficial mutation (selection coefficient = 0.5; dominance coefficient = 0.5) to the gene expressed in the sporophyte. 

This simulation was included in Sorojsrisom et al. (2022), alongside a more complex simulation investigating pattens of diversity across the genome resulting from targeted differential selection on each life stage (see this other jupyter notebook).

#### Make the Burn-in

In [6]:
with shadie.Model() as bryomono_burnin:
    # init the model
    bryomono_burnin.initialize(chromosome=chrom, 
                     sim_time = 10000, 
                     mutation_rate = 1e-6,
                     recomb_rate = 1e-07,
                     file_out = "single-site-trees/burnin-bryo-mono.trees")
        
    # set model parameters 
    bryomono_burnin.reproduction.bryophyte_monoicous(
        spo_pop_size=3000, 
        gam_pop_size=3000,
        gam_clone_rate=0.2,
        gam_clones_per=1,
        spo_spores_per=10,
        gam_archegonia_per=10,)
    
#Uncomment line below to view the script
#print(bryomono_burnin)

In [7]:
#Uncomment line below to run the simulation

#bryomono_burnin.run()

#### Custom Scripts for Adding Mutations
The scripts below are written in Eidos, SLiM's native language, and introduce a single beneficial mutation to the middle of the chromosome. 

In [17]:
# These are the SLiM scripts that will add a mutation at a given time. 

add_mut_gam = """
2000 late() {
    num_genomes = length(p0.genomes);
	list_genomes = 1:num_genomes ;
	even_genomes= (p0.genomes)[list_genomes % 2 == 1];
	target = sample(even_genomes, 1);
	target.addNewDrawnMutation(m3, 2500000);
}
"""

add_mut_spo = """
2001 late() {
    target = sample(p1.genomes, 1);
    target.addNewDrawnMutation(m4, 2500000);
}
"""

#### Set up the simulation in `shadie`

In [32]:
with shadie.Model() as bryomono_gam:
    # init the model
    bryomono_gam.initialize(chromosome=chrom, 
                     sim_time = 5000, 
                     mutation_rate = 1e-6,
                     recomb_rate = 1e-07,
                     file_in= "single-site-trees/burnin-bryo-mono.trees",
                     file_out = "singlemutgam-bryo-mono.trees")
    
    # add custom script which adds a single beneficial mutation 
    # to each gene at t=6000
    bryomono_gam.custom(
        scripts=add_mut_gam,
        comment = "add beneficial mutation to gametophytes at t=2000")
        
    # set model parameters 
    bryomono_gam.reproduction.bryophyte_monoicous(
        spo_pop_size=500, 
        gam_pop_size=500,
        gam_clone_rate=0.2,
        gam_clones_per=1,
        spo_spores_per=10,
        gam_archegonia_per=10,)
    
#Uncomment line below to view the script
#print(bryomono_gam.script)

/Users/elissa/code/git/pyslim/pyslim/slim_tree_sequence.py:902: UserWarning: This is an version 0.7 SLiM tree sequence. When you write this out, it will be converted to version 0.6.
  warnings.warn("This is an version {} SLiM tree sequence.".format(file_version) +


In [24]:
with shadie.Model() as bryomono_spo:
    # init the model
    bryomono_spo.initialize(chromosome=chrom, 
                     sim_time = 5000, 
                     mutation_rate = 1e-6,
                     recomb_rate = 1e-07,
                     file_in= "single-site-trees/burnin-bryo-mono.trees",
                     file_out = "singlemutspo-bryo-mono.trees")
    
    # add custom script which adds a single beneficial mutation 
    # to each gene at t=6000
    bryomono_spo.custom(
        scripts=add_mut_spo,
        comment = "add beneficial mutation to gametophytes at t=2001")
        
    # set model parameters 
    bryomono_spo.reproduction.bryophyte_monoicous(
        spo_pop_size=500, 
        gam_pop_size=500,
        gam_clone_rate=0.2,
        gam_clones_per=1,
        spo_spores_per=10,
        gam_archegonia_per=10,)
    
#Uncomment line below to view the script
#print(bryomono_spo.script)

/Users/elissa/code/git/pyslim/pyslim/slim_tree_sequence.py:902: UserWarning: This is an version 0.7 SLiM tree sequence. When you write this out, it will be converted to version 0.6.
  warnings.warn("This is an version {} SLiM tree sequence.".format(file_version) +


### Running the Simulations
You can run the simulations individually using `.run*()`. You can also used `ProcessPoolExecutor` to run the simulations in parallel. You can also set up the simulations to run serially and save out with different filenames using a for-loop. 

In [12]:
from concurrent.futures import ProcessPoolExecutor
import numpy as np

#### Running the simulations individually

In [33]:
#Uncomment the lines below to run simulations individually

#bryomono_gam.run(seed=np.random.randint(2**31))
#bryomono_spo.run(seed=np.random.randint(2**31))

#### Running the simulations in parallel

In [21]:
with ProcessPoolExecutor(2) as pool:
    for model in [bryomono_gam, bryomono_spo]:
        kwargs = {"seed": np.random.randint(2**31)}
        pool.submit(model.run, **kwargs)

AttributeError: 'str' object has no attribute 'run'

#### Using a for-loop to set up many simulations to run serially

In [ ]:
#FOR-LOOP EXAMPLE


## Analysis

In [46]:
for i in range(1,5):
    filename = f"bryomono-trees/burnin-bryo-mono{i}.trees"
    print(filename)

bryomono-trees/burnin-bryo-mono1.trees
bryomono-trees/burnin-bryo-mono2.trees
bryomono-trees/burnin-bryo-mono3.trees
bryomono-trees/burnin-bryo-mono4.trees


In [ ]:
## Using the `PostSim` Module

In [1]:
import pyslim

In [7]:
gam1 = pyslim.load("single-site-trees/singlemutgam-bryo-mono.trees")
gam2 = pyslim.load("single-site-trees/singlemutgam-bryo-mono1.trees")
gam3 = pyslim.load("single-site-trees/singlemutgam-bryo-mono2.trees")
sims = (gam1, gam2, gam3)

/Users/elissa/code/git/pyslim/pyslim/slim_tree_sequence.py:902: UserWarning: This is an version 0.7 SLiM tree sequence. When you write this out, it will be converted to version 0.6.
  warnings.warn("This is an version {} SLiM tree sequence.".format(file_version) +
/Users/elissa/code/git/pyslim/pyslim/slim_tree_sequence.py:902: UserWarning: This is an version 0.7 SLiM tree sequence. When you write this out, it will be converted to version 0.6.
  warnings.warn("This is an version {} SLiM tree sequence.".format(file_version) +
/Users/elissa/code/git/pyslim/pyslim/slim_tree_sequence.py:902: UserWarning: This is an version 0.7 SLiM tree sequence. When you write this out, it will be converted to version 0.6.
  warnings.warn("This is an version {} SLiM tree sequence.".format(file_version) +


In [8]:
for i in sims:    
    mtypes = []
    for mut in i.mutations():
        for i in range(0,len(mut.metadata['mutation_list'])):
            this_type = mut.metadata['mutation_list'][i]['mutation_type']
            if this_type not in mtypes:
                mtypes.append(this_type)
    print(mtypes)

[6]
[6]
[6]


In [9]:
gam1.mutation(6)

Mutation(id=6, site=6, node=14400, derived_state='478761815', parent=-1, metadata={'mutation_list': [{'mutation_type': 6, 'selection_coeff': 0.0, 'subpopulation': 1, 'slim_time': 9935, 'nucleotide': -1}]}, time=5066.0)

In [38]:
bryomono_gam1 = "single-site-trees/singlemutgam-bryo-mono1.trees"
bryomono_gam2 = "single-site-trees/singlemutgam-bryo-mono2.trees"

In [39]:
post1 = shadie.postsim.OneSim(
    bryomono_gam1,
    chromosome=chrom, 
    mut=5e-7,
    recomb=1e-7, 
    recapitate=False, 
    ancestral_Ne=3000, 
    add_neutral_mutations=False,
)

post2 = shadie.postsim.OneSim(
    bryomono_gam2,
    chromosome=chrom, 
    mut=5e-7,
    recomb=1e-7, 
    recapitate=False, 
    ancestral_Ne=3000, 
    add_neutral_mutations=False,
)

post1.tree_sequence

/Users/elissa/code/git/pyslim/pyslim/slim_tree_sequence.py:902: UserWarning: This is an version 0.7 SLiM tree sequence. When you write this out, it will be converted to version 0.6.
  warnings.warn("This is an version {} SLiM tree sequence.".format(file_version) +
/Users/elissa/code/git/pyslim/pyslim/slim_tree_sequence.py:902: UserWarning: This is an version 0.7 SLiM tree sequence. When you write this out, it will be converted to version 0.6.
  warnings.warn("This is an version {} SLiM tree sequence.".format(file_version) +


In [40]:
post2.tree_sequence

'single-site-trees/singlemutgam-bryo-mono1.trees'

In [41]:
post1.draw_tree_sequence(max_trees=10);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1500.0px" height="425.0px" viewBox="0 0 1500.0 425.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0cc28e6309414e888cf6e33dea5fa892"> name: None
interval: (0, 1000000)
ElementType: g5
coding: True 0 200000 400000 600000 800000 1000000 idx: 0
interval: (0 - 11237)
tmrca: 15000.0 idx: 1
interval: (11237 - 19357)
tmrca: 15000.0 idx: 2
interval: (19357 - 21632)
tmrca: 15000.0 idx: 3
interval: (21632 - 22426)
tmrca: 15000.0 idx: 4
interval: (22426 - 24727)
tmrca: 15000.0 idx: 5
interval: (24727 - 25157)
tmrca: 15000.0 idx: 6
interval: (25157 - 27467)
tmrca: 15000.0 idx: 7
interval: (27467 - 28758)
tmrca: 15000.0 idx: 8
interval: (28758 - 32223)
tmrca: 15000.0 idx: 9
interval: (32223 - 33139)
tmrca: 15000.0 p0-9 p0-8 p0-6 p0-4 p0-3 p0-5 p0-0 p0-7 p0-2 p0-1 id: 0
pos: 0
time: 13074
mtype: 6 id: 1
pos: 1
time: 4818
mtype: 6 id: 2
pos: 2
time: 2421
mtype: 6 id: 3
pos: 3
time: 11457
mtype: 6 id: 4
pos: 4
time: 10565
mtype: 6 id: 5
pos: 5
time: 11024
mtype: 6 id: 6
pos: 6
time: 213
mtype: 6 id: 7
pos: 7
time: 9645
mtype: 6 id: 8
pos: 8
time: 8417
mtype: 6 id: 9
pos: 9
time: 896
mtype: 6 id: 10
pos: 10
time: 14919
mtype: 6 id: 11
pos: 11
time: 8453
mtype: 6 id: 12
pos: 12
time: 10146
mtype: 6 id: 13
pos: 13
time: 1581
mtype: 6 id: 14
pos: 14
time: 7226
mtype: 6 id: 15
pos: 15
time: 8504
mtype: 6 id: 16
pos: 16
time: 5684
mtype: 6 id: 17
pos: 17
time: 8775
mtype: 6 id: 18
pos: 18
time: 1585
mtype: 6 id: 19
pos: 19
time: 4274
mtype: 6 id: 20
pos: 20
time: 62
mtype: 6 id: 21
pos: 21
time: 1154
mtype: 6 id: 22
pos: 22
time: 13640
mtype: 6 id: 23
pos: 23
time: 12352
mtype: 6 id: 24
pos: 24
time: 12741
mtype: 6 id: 25
pos: 25
time: 4036
mtype: 6 id: 26
pos: 26
time: 14639
mtype: 6 id: 27
pos: 27
time: 11739
mtype: 6 id: 28
pos: 28
time: 4354
mtype: 6 id: 29
pos: 29
time: 14963
mtype: 6 id: 30
pos: 30
time: 14023
mtype: 6 id: 31
pos: 31
time: 13454
mtype: 6 id: 32
pos: 32
time: 8487
mtype: 6 id: 33
pos: 33
time: 258
mtype: 6 id: 34
pos: 34
time: 4861
mtype: 6 id: 35
pos: 35
time: 12565
mtype: 6 id: 36
pos: 36
time: 8238
mtype: 6 id: 37
pos: 37
time: 14318
mtype: 6 id: 38
pos: 38
time: 385
mtype: 6 id: 39
pos: 39
time: 10407
mtype: 6 id: 40
pos: 40
time: 1991
mtype: 6 id: 41
pos: 41
time: 9417
mtype: 6 id: 42
pos: 42
time: 12161
mtype: 6 id: 43
pos: 43
time: 818
mtype: 6 id: 44
pos: 44
time: 440
mtype: 6 id: 45
pos: 45
time: 13480
mtype: 6 id: 46
pos: 46
time: 5433
mtype: 6 id: 47
pos: 47
time: 11583
mtype: 6 id: 48
pos: 48
time: 10141
mtype: 6 id: 49
pos: 49
time: 2997
mtype: 6 id: 50
pos: 50
time: 11751
mtype: 6 id: 51
pos: 51
time: 8481
mtype: 6 id: 52
pos: 52
time: 3679
mtype: 6 id: 53
pos: 53
time: 12
mtype: 6 id: 54
pos: 54
time: 10208
mtype: 6 id: 55
pos: 55
time: 2913
mtype: 6 id: 56
pos: 56
time: 10496
mtype: 6 id: 57
pos: 57
time: 705
mtype: 6 id: 58
pos: 58
time: 5655
mtype: 6 id: 59
pos: 59
time: 4401
mtype: 6 id: 60
pos: 60
time: 198
mtype: 6 id: 61
pos: 61
time: 12231
mtype: 6 id: 62
pos: 62
time: 5927
mtype: 6 id: 63
pos: 63
time: 1538
mtype: 6 id: 64
pos: 64
time: 67
mtype: 6 id: 65
pos: 65
time: 5892
mtype: 6 id: 66
pos: 66
time: 13052
mtype: 6 id: 67
pos: 67
time: 4905
mtype: 6 id: 68
pos: 68
time: 219
mtype: 6 id: 69
pos: 69
time: 1692
mtype: 6 id: 70
pos: 70
time: 3289
mtype: 6 id: 71
pos: 71
time: 10037
mtype: 6 id: 72
pos: 72
time: 10111
mtype: 6 id: 73
pos: 73
time: 7795
mtype: 6 id: 74
pos: 74
time: 13844
mtype: 6 id: 75
pos: 75
time: 7944
mtype: 6 id: 76
pos: 76
time: 14491
mtype: 6 id: 77
pos: 77
time: 14378
mtype: 6 id: 78
pos: 78
time: 13760
mtype: 6 id: 79
pos: 79
time: 7927
mtype: 6 id: 80
pos: 80
time: 12251


## All model scripts